In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## static object
state update equation

In [ ]:
## generate measurement data
miu=5
sigma=2
measureTimes=40
Z=np.random.normal(miu,sigma,measureTimes)

In [ ]:
Z

In [ ]:
X=np.zeros(measureTimes+1)
X[0]=miu
for measure in range(measureTimes):
    n=measure+1
    gain=1/n
    X[n]=X[n-1]+gain*(Z[measure]-X[n-1])


In [ ]:
plt.plot(X)
plt.plot(Z,marker='o')
plt.plot([0,measureTimes],[miu,miu])
plt.xlabel("measure times")
plt.ylabel("estimate value")
plt.legend(['estimate','measure','real'])
plt.title("kalman filter for static object")
plt.show()

## $\alpha-\beta$ filter
(g-h filter) for dynamic object (state update)    
state extrapolation equation

### situation design 
a vehicle go at 10m/s, initial site is 0m, and the measure tool is unbiased, but the measurements include random noise(gaussian distribution) 

In [ ]:
measureTimes=40
velocity=10 #m/s
interval=0.1 #s 
Narray=np.asarray([n for n in range(measureTimes)])
preciseX=Narray*velocity*interval

In [ ]:
Narray
preciseX

In [ ]:
## generate measurements
Z=np.zeros(measureTimes)
sigma=interval*velocity
for time in range(measureTimes):
    Z[time]=np.random.normal(preciseX[time],sigma,1)

In [ ]:
Z

## question
how to make sure the initial value?

In [ ]:
X=np.zeros(measureTimes+1)
V=np.zeros(measureTimes+1)
for measure in range(measureTimes):
    n=measure+1
    estimateX=X[n-1]+interval*V[n-1]
    alpha=0.2
    beta=0.1

    X[n]=estimateX+alpha*(Z[measure]-estimateX)
    V[n]=V[n-1]+beta*(Z[measure]-estimateX)/interval
    #print(V)


alpha can not decrease with time anymore, for the state is updating all the time

if update state with large number, the estimation is not so stable
how about minimize the interval

!!! important the error times by kalman gain is not new value, but estimate value according to prediction

In [ ]:
plt.plot(X)
plt.plot(Z,marker='o')
plt.plot(preciseX)
plt.plot(V)
plt.plot([0,measureTimes],[velocity,velocity])
plt.xlabel("measure times")
plt.ylabel("estimate value")
plt.legend(['measure','estimate x','real x','estimate v','real v'])
plt.title("kalman filter for static object")
plt.show()

In [ ]:
V

In [ ]:
X